In [77]:
!pip install gdown
import pandas as pd
#import files from google drive
import gdown

url = 'https://drive.google.com/uc?id=1-WZKE5xHw-3m_SL_PtOgwkzdFROIWqih'
output = 'reviews.csv'
gdown.download(url, output, quiet=False)

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
Downloading...
From: https://drive.google.com/uc?id=1-WZKE5xHw-3m_SL_PtOgwkzdFROIWqih
To: C:\Users\KIIT\Desktop\ML\reviews.csv
100%|███████████████████████████████████████████████████████████████████████████████| 301M/301M [01:41<00:00, 2.98MB/s]


'reviews.csv'

In [78]:
raw_df = pd.read_csv(output)
#we will only look at reviews where score is > 0
raw_df = raw_df[raw_df['Score'] > 0]
df = raw_df[:2000]

In [79]:
df = raw_df[:2000]
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [80]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from collections import defaultdict
lemmatizer = WordNetLemmatizer()

tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

def preprocess_with_lemmatization(document):
  tokens = nltk.word_tokenize(document.lower())
  tagged_tokens = nltk.pos_tag(tokens)
  lemmas = []
  for token, pos in tagged_tokens:
    lemmatizer_tag = tag_map[pos[0]]
    lemma = lemmatizer.lemmatize(token, pos=lemmatizer_tag)
    lemmas.append(lemma)
  return lemmas

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [81]:
#default dict starts counting from 0 when its a new word
from collections import defaultdict

#no of times a word appears in a five star review
five_star_counts = defaultdict(int)
one_star_counts = defaultdict(int)
# total_counts = defaultdict(int)
total_counts = defaultdict(lambda: 10)
# lambda: 10 finds word in the document which occurs at least 10 times


#zip helps in iterating through the two lists at the same time
for text, score in zip(df['Text'], df['Score']):
  tokens = preprocess_with_lemmatization(text)
  for token in tokens:
    if score == 5:
      five_star_counts[token] += 1
    elif score == 1:
      one_star_counts[token] += 1
    total_counts[token] += 1

In [82]:
sorted(five_star_counts.keys(),key=lambda token: five_star_counts[token], reverse=True)[:10]

['.', 'be', 'the', ',', 'i', 'and', 'a', 'to', 'it', 'of']

In [83]:
sorted(one_star_counts.keys(), key=lambda token: one_star_counts[token], reverse=True)[:10]

['.', 'be', 'the', 'i', ',', 'and', 'a', 'it', 'to', 'of']

In [84]:
# TF-IDF
# Term
# Frequency
# Inverse
# Document 
# Frequency
# We count the number of times a word appears and divide it with the nummber of times it appears anywhere

In [85]:
# finding out good tokens
# We categorize unique words only available on tokens in five star reviews
five_star_normalized = {}

for token in five_star_counts:
    normalized_five_star_count = five_star_counts[token] / total_counts[token]
    five_star_normalized[token] = normalized_five_star_count
    
good_tokens = sorted(five_star_normalized.keys(),key=lambda token: five_star_normalized[token], reverse=True)[:10]
good_tokens

['highly',
 'dog',
 'great',
 'love',
 'delicious',
 'wonderful',
 'recommend',
 '--',
 'weight',
 'ever']

In [86]:
five_star_normalized = {}

for token in five_star_counts:
  normalized_five_star_count = five_star_counts[token] / total_counts[token]
  five_star_normalized[token] = normalized_five_star_count

one_star_normalized = {}

for token in one_star_counts:
  normalized_one_star_count = one_star_counts[token] / total_counts[token]
  one_star_normalized[token] = normalized_one_star_count

good_tokens = sorted(five_star_normalized.keys(), key=lambda token: five_star_normalized[token], reverse=True)[:20]
bad_tokens = sorted(one_star_normalized.keys(), key=lambda token: one_star_normalized[token], reverse=True)[:20]

good_tokens,bad_tokens

(['highly',
  'dog',
  'great',
  'love',
  'delicious',
  'wonderful',
  'recommend',
  '--',
  'weight',
  'ever',
  'thank',
  'snack',
  'fast',
  'our',
  'best',
  'thanks',
  'perfect',
  'calorie',
  'she',
  '!'],
 ['alive',
  'frosting',
  'aware',
  'return',
  'awful',
  'throw',
  'horrible',
  'donut',
  'disappointed',
  'cacao',
  'garbage',
  'maltitol',
  'picture',
  'stale',
  'frost',
  'change',
  'label',
  'terrible',
  'cancel',
  'clearly'])

In [87]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [88]:
def predict_stars(document):
    # We'll start with "neutral"
    sentiment = 0
    # Our default preprocessing will be just splitting along spaces, but
    # we can pass in a custom preprocess script later if we want
    tokens = preprocess_with_lemmatization(document)

    # We loop through all the tokens in our document
    for token in tokens:
        # If the token is one of our "good" ones, we'll add 1 to the sentiment
        if token in good_tokens:
            sentiment += 1
        # If the token is one of our "bad" ones, we'll subtract 1 from the sentiment
        if token in bad_tokens:
            sentiment -= 1
    
    if sentiment > 1:
      return 5
    elif sentiment == 1:
      return 4
    elif sentiment == 0:
      return 3
    elif sentiment == -1:
      return 2
    else:
      return 1

df['Prediction'] = df['Text'].apply(predict_stars)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_135440\1893550487.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prediction'] = df['Text'].apply(predict_stars)


In [89]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Prediction
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,5
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,5
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,3
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,5


In [90]:
correct_cases = sum(df['Prediction'] == df['Score'])
total_cases = len(df['Prediction'])
accuracy = correct_cases / total_cases 
accuracy

# 22.00  -->  33.34  -->  52.85 for first 1000
# if you increase the number of test cases to 2000 the accuracy should probably increase
# human prediction should be about 20.00

0.5285

In [91]:
corpus = [
          "You like dogs",
          "You like cats",
          "You love dogs",
          "You love cats",
]

vocab = set()

for doc in corpus:
  tokens = preprocess_with_lemmatization(doc)
  vocab.update(tokens)
    
#this will update all of our individual tokens in the set
#we then turned our vocab to a list and then sorted it
vocab = list(vocab)
vocab.sort()
print(vocab)

['cat', 'dog', 'like', 'love', 'you']


In [92]:
# D: number of docs in our corpus
# T: maximum number of tokens in any document (T = 100)

word_to_index = {}
for i, word in enumerate(vocab):
  word_to_index[word] = i

#enumerate helps in going through the list of vocabs and gives the list of indices
#we associate indices to every word present in the vocab

vectors = []
for doc in corpus: # D
  tokens = preprocess_with_lemmatization(doc)
  vector = [0] * len(vocab)
  for token in tokens: # T
    index = word_to_index[token]
    vector[index] += 1
  vectors.append(vector)

# O(D)
#maximum number of tokens in a word remains constant for which time complexity remains as O(D)

In [93]:
#unpacks the vector and prints with a seprator \n
print(word_to_index)
print()
print(*corpus,sep="\n")
print()
print(*vectors, sep="\n")
#after we converted our word_to_index we got five unique words so we vectorize it
#In the first document we get 'you','like','dogs' which are labelled as 4 2 1 so we put 1 on those places rest as 0

{'cat': 0, 'dog': 1, 'like': 2, 'love': 3, 'you': 4}

You like dogs
You like cats
You love dogs
You love cats

[0, 1, 1, 0, 1]
[1, 0, 1, 0, 1]
[0, 1, 0, 1, 1]
[1, 0, 0, 1, 1]


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
#count vectorization 
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(corpus)
#what fir_transform does is what we did manually fit actually finds unique words and appends it to the vocab
#trnasform helps in transforming the words into vector notation
print(vectors.toarray())

[[0 1 1 0 1]
 [1 0 1 0 1]
 [0 1 0 1 1]
 [1 0 0 1 1]]


In [103]:
df = raw_df[:10000]
training_df = raw_df[:9000]
testing_df = raw_df[9000:10000]

In [104]:
vectorizer = CountVectorizer()
training_vectors = vectorizer.fit_transform(training_df['Text'])

In [105]:
first_vector = training_vectors.toarray()[0]
print(first_vector)
print(first_vector.size)

[0 0 0 ... 0 0 0]
18141


In [106]:
testing_vectors = vectorizer.transform(training_df['Text'])
first_testing_vector = testing_vectors.toarray()[0]
print(first_testing_vector)
print(first_testing_vector.size)

[0 0 0 ... 0 0 0]
18141


In [107]:
training_labels = training_df['Score']
testing_labels = testing_df['Score']

training_labels[:10]

9000    5
9001    1
9002    2
9003    5
9004    5
9005    4
9006    4
9007    3
9008    5
9009    3
Name: Score, dtype: int64

In [111]:
from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression()
lr_classifier.fit(training_vectors,training_labels)

C:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [120]:
lr_classifier.score(training_vectors,training_labels)

0.9462222222222222